This notebook loads data from a CSV, AVRO and JSON files and combines them in a single deduplicated dataframe and answers some questions.

URL(https://github.com/implydata/candidate-exercises-public/blob/master/Customer%20Success/SA/DataEngineeringProject/Applicant/) for the challenge and datasets below.

The notebook uses the following 
python <= 3.10 
Jupyter Notebook
Pandas *
fastavro *

Challenge summary replicated from the Github URL.

There are three source files. CityListA.json, CityListB.avro, and CityListC.csv. They each contain data in three columns:

Schema for the files
name:string
code:string
Population:long

The goal is to combine the files, eliminating any duplicates and write to a single .CSV file sorted alphabetically by the city name. 

Q1: What is the count of all rows?
Q2: What is the city with the largest population?
Q3: What is the total population of all cities in Brazil (CountryCode == BRA)?
Q4: What changes could be made to improve your program's performance.
Q5: How would you scale your solution to a much larger dataset (too large for a single machine to store)?

In [6]:
import pandas as pd
'''
In this snippet, a file handle for CSV is created and used as an iterator to read 10K records at a time. 
While we could append the rows into dataframe inside the for loop, this impacts performance negatively, because it doubles the memory requirements.
This is because the data frame which is being appended to as well as the chunk is typecasted as dataframe. 
A simpler approach is to build a list in memory and append all the records inside the loop.
Post loop, convert the list to dataframe. 
'''
csv_handle = pd.read_csv('./Datasets/CityList.csv', chunksize=10000)
csv_data_lst = []
for i in csv_handle:
    csv_data_lst.append(i)

csv_df = pd.concat(csv_data_lst)
csv_df.head()
csv_df.shape    

(1000, 3)

In [7]:
import pandas as pd
'''
Uses the build-in read_json method from the pandas package. While this is OK for small files, this is not recommended for large file due to schema replication per record.
An Avro format is much better. 
'''
json_df = pd.read_json('./Datasets/CityListA.json')
print(json_df.shape)
json_df.head() 

(1000, 3)


,Name,CountryCode,Population
0,Kabul,AFG,1780000
1,Qandahar,AFG,237500
2,Amsterdam,NLD,731200
3,Utrecht,NLD,234323
4,Eindhoven,NLD,201843


In [8]:
from fastavro import writer, reader, parse_schema
import pandas as pd
'''
Importing pandas multiple times is redundant.
Using the fastavro package.
I have defined a schema for the file and validated using parse_schema().

Like the CSV file approach, I read chunks of Avro file, and build a list inside the loop.
Post loop, I can cast the list as a dataframe.
'''
av_schema = {
    "type" : "record",
    "namespace" : "exercise_avro",
    "name" : "citylistB",
    "fields" :[
        {"name": "name", "type": "string" },
        {"name": "code", "type": "string" },
        {"name": "Population", "type": "long" },
    ]
}

av_schema_check = parse_schema(av_schema)
print(av_schema_check)

av_data_lst = []
with open('./Datasets/CityListB.avro', 'rb') as city_av:
    for i in reader(city_av):
        av_data_lst.append(i)

print(len(av_data_lst)) 

avro_df = pd.DataFrame(av_data_lst)
print(avro_df.shape)
avro_df.head()

{'type': 'record', 'name': 'exercise_avro.citylistB', 'fields': [{'name': 'name', 'type': 'string'}, {'name': 'code', 'type': 'string'}, {'name': 'Population', 'type': 'long'}], '__fastavro_parsed': True, '__named_schemas': {'exercise_avro.citylistB': {'type': 'record', 'name': 'exercise_avro.citylistB', 'fields': [{'name': 'name', 'type': 'string'}, {'name': 'code', 'type': 'string'}, {'name': 'Population', 'type': 'long'}]}}}
583
(583, 3)


,Name,CountryCode,Population
0,Mazar-e-Sharif,AFG,127800
1,Groningen,NLD,172701
2,Arnhem,NLD,138020
3,Haarlemmermeer,NLD,110722
4,Alkmaar,NLD,92713


In [9]:
'''
Using pandas concat() method to combine the 3 datarames and exporting a file to allow visual inspection.
Using drop_duplicates() to remove record duplicates and exporting to a file for inspection. 
Using sort_values() method to sort data by City Names, alphabetically.
Memory analysis for dataframe columns, e.g. CountryCode String --> Categorical
'''
combined_df1 = pd.concat([csv_df,json_df,avro_df])
print(combined_df1.shape)
combined_df1.to_csv('combined_b4dedup.csv')

combined_df1.drop_duplicates(inplace=True)
combined_df1.sort_values(by=['Name'], ascending=False, inplace=True)
combined_df1.to_csv('combined_a4dedup.csv')

print(combined_df1.head(10))
print(combined_df1.shape)
print(combined_df1.memory_usage(deep=True))
print(combined_df1['CountryCode'].astype('category').memory_usage())

(2583, 3)
                        Name CountryCode  Population
235                   Å umen         BGR       94686
993                  Å ostka         UKR       90000
699                Å iauliai         LTU      146563
523                   Å ahty         RUS      221800
967                   Ã‡orum         TUR      145495
129  Ãguas Lindas de GoiÃ¡s         BRA       89200
766                 al-Zarqa         JOR      389815
910                 al-Tuqba         SAU      125700
907                  al-Taif         SAU      416100
587           al-Sulaymaniya         IRQ      364096
(2083, 3)
Index           16664
Name           144414
CountryCode    124980
Population      16664
dtype: int64
26390


In [10]:
'''
Exercise Questions
Q1: What is the count of all rows?
Q2: What is the city with the largest population?
Q3: What is the total population of all cities in Brazil (CountryCode == BRA)?
'''
print(f"What is the count of all rows? \n",len(combined_df1.index))
print(f"What is the city with the largest population? \n", combined_df1[combined_df1['Population'] == combined_df1['Population'].max()])
print(f"What is the total population of all cities in Brazil (CountryCode == BRA)? \n", combined_df1.loc[combined_df1.CountryCode == 'BRA', 'Population'].sum())

What is the count of all rows? 
 2083
What is the city with the largest population? 
                 Name CountryCode  Population
439  Mumbai (Bombay)         IND    10500000
What is the total population of all cities in Brazil (CountryCode == BRA)? 
 55955012


Q4: What changes could be made to improve your program's performance?\
A4: Choice of better file formats and datatypes. AVRO over JSON, and convert and store CountryCode as a Categorical.  

Q5: How would you scale your solution to a much larger dataset (too large for a single machine to store)?\
A5: we could use dask library to load large files in parallel, along with Spark to hold teh distributed set in memory till it is committed to a durable storage.
